In [2]:
from asyncore import read
import spacy
from spacy import displacy
from utils.utils import count_occurrences, get_entities, search_with_google, wiki_content
import json

nlp = spacy.load('it_core_news_sm')

with open(f'assets/test_sentences.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

ModuleNotFoundError: No module named 'utils'

In [2]:
sentence_list = list(doc.sents) 

# Opening JSON file
f = open('./assets/italian_cities.json')
italian_cities = json.load(f)
f.close()
# Count occurrences
counter = count_occurrences(doc, italian_cities, "name")  

# Max occurrence
context = max(counter, key=counter.get) 
print(context)

# Get entities without duplicates
searchable_entities = get_entities(doc, counter)

print(searchable_entities)
#print(searchable_entities)

# Search addresses with Google
search_with_google(searchable_entities, context)

Torino
['Novecento', 'Italia', 'Ottocento', 'Novecento.', 'Impresa Porcheddu', 'Esposizione generale italiana', 'Borgo Medievale', 'Vetreria Albano&Macario', 'Terrazza Solferino', 'Mobilificio Torinese F. Cesare Gandolfo', 'Albergo Rocciamelone di', 'Francia', 'Villino Raby', 'Casa Fenoglio-Lafleur', 'Casa Rossi-Galateri', 'Passalacqua', 'Casa Girardi', 'Cibrario 54.', 'Crocetta', 'Casa Maffei']
https://google.it/search?q=Novecento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Italia+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Ottocento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Novecento.+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Impresa+Porcheddu+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Esposizione+generale+italiana+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Borgo+Medievale+Torino&hl=it
Address:  None
Activity:  None
ht

In [3]:
print(spacy.explain("LOC"))

Non-GPE locations, mountain ranges, bodies of water


In [12]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="PoI_geocoder")

location = geolocator.geocode("Petronas Torino", exactly_one = False, geometry="geojson" )

#print(location)
for loc in location: 
    print(loc.address)
    #Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
    print((loc.latitude, loc.longitude))
    #(40.7410861, -73.9896297241625)
    print(loc.raw)
    #{'place_id': '9167009604', 'type': 'attraction', ...}

Petronas Lubricants International, Villastellone, Torino, Piemonte, 10029, Italia
(44.93258615, 7.754597116824563)
{'place_id': 172708711, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 278793093, 'boundingbox': ['44.927963', '44.9374412', '7.7516704', '7.7598834'], 'lat': '44.93258615', 'lon': '7.754597116824563', 'display_name': 'Petronas Lubricants International, Villastellone, Torino, Piemonte, 10029, Italia', 'class': 'landuse', 'type': 'industrial', 'importance': 0.41, 'geojson': {'type': 'Polygon', 'coordinates': [[[7.7516704, 44.9281175], [7.752023, 44.927963], [7.7562041, 44.9321458], [7.7565046, 44.9331033], [7.757027, 44.9330265], [7.7572645, 44.9332762], [7.7573255, 44.9335356], [7.7582347, 44.9344194], [7.7584247, 44.9343617], [7.758886, 44.9347988], [7.7595645, 44.936062], [7.7598087, 44.9359899], [7.7598834, 44.9361965], [7.755935, 44.9372506], [7.7534445, 44.9374412], [7.7534818, 44.9367868], [7.753492, 

In [1]:
import spacy
from spacy import displacy

nlp = spacy.load('it_core_news_sm')
sentence = "A Torino nei primi anni ’90 è stata fondata ed ha sede la Federazione Italiana Hit Ball (FIHB) che dal 1992 organizza il campionato nazionale. Altri sport praticati in città comprendono baseball, curling, hockey, pallanuoto, pallamano, scherma, tennis tavolo, tiro con l'arco, bocce e pallone elastico.  Eventi sportivi Per quanto riguarda gli eventi sportivi, il maggiore vanto della città è l'aver ospitato la XX edizione dei Giochi olimpici invernali, svoltisi il 10-26 febbraio del 2006, seguiti a marzo dalle Paralimpiadi."
                             
doc = nlp(sentence)

displacy.render(doc, style="dep", jupyter=True)

displacy.render(doc, style="ent", jupyter=True)
                